#Graphs & Social Networks &mdash; lab material
This notebook contains a definition of a python class *Graph*, containing functions described during labs.

Package import.

In [27]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from random import random, seed
from copy import deepcopy
import time
from collections import deque

## Simple functions from the first classes
This is left just as an example (to compare the behaviour).

In [28]:
def print_matrix(vertices, matrix):
  """
  Printing a graph given by adjacency matrix
  """
  n = len(matrix)
  if (vertices is not None) and (len(vertices) == n):
    vv = vertices
  else:
    vv = range(1, n+1)
  for i in range(n):
    print(vv[i], ":", end="")
    for j in range(n):
      if matrix[i, j]:
        print(" ", vv[j], end="")
    print("")

def print_dict(graph):
  """
  Printing of a graph (given as a dictionary/neighbouring list)
  """
  for v in graph:
    print(v, ":", end="")
    for u in graph[v]:
      print(" ", u, end="")
    print("")

## Class *Graph*

In [29]:
class Graph:
    def __init__(self, graph=None):
        if graph is None:
            graph = {}
        self.graph = graph

    # dict initializer
    @classmethod
    def from_dict(cls, graph):
        return cls(graph)

    # array initializer
    @classmethod
    def from_matrix(cls, matrix, vertices = None):
        if (vertices is None) or (len(vertices) != len(matrix)):
            vertices = [*range(1, len(matrix) + 1)]
        return cls.from_dict(cls._matrix_to_dict(matrix, vertices))

    # two private methods matrix <-> dictionaries
    def _matrix_to_dict(matrix, vertices: list) -> dict:
        """
        Converts a graph given as an adjacency matrix to a graph in dict form.
        """
        res_dict = {}
        for i, v in enumerate(vertices):
            neighbours = [vertices[j] for j, edge in enumerate(matrix[i]) if edge]
            res_dict[v] = neighbours
        return res_dict

    def _dict_to_matrix(self, _dict: dict) -> np.array:
        """
        Converts a graph in dict form to its adjacency matrix.
        """
        n = len(_dict)
        vertices = [*_dict.keys()]
        matrix = np.zeros(shape = (n, n), dtype=int)
        for u,v in [
            (vertices.index(u), vertices.index(v))
            for u, row in _dict.items() for v in row
        ]:
            matrix[u][v] += 1
        return matrix

    def vertices(self) -> list:
        """
        Returns list of vertices of the graph.
        """
        return [*self.graph.keys()]

    def matrix(self) -> np.array:
        """
        Returns the adjacency matrix of the graph.
        """
        return self._dict_to_matrix(self.graph)

    # redefinition of print for objects of class Graph
    def __str__(self):
        res = ""
        for v in self.graph:
            res += f"{v}:"
            for u in self.graph[v]:
                res += f" {u}"
            res += "\n"
        return res

    def plot(self):
      """
      Plots the graph using methods from networkx package.
      """
      G = nx.Graph(self.graph)
      pos = nx.spring_layout(G)
      nx.draw(G, pos, with_labels=True)
      plt.show()


    # The following is for free thanks to the above
    def to_neighbourlist(self, filename: str):
        """
        Saves a graphs to a text file as a neighbour dict.\n
        Filename is a file path.
        """
        file = open(filename, "w")  # open textfile for writing
        file.write(str(self))
        file.close()

    # Modyfying graphs
    def add_vertex(self, vertex):
        """
        Adds a new vertex to the graph.
        """
        if vertex not in self.graph:
            self.graph[vertex] = []

    def del_vertex(self, vertex):
        """
        Removes a vertex from the graph.
        """
        if vertex in self.graph:
            self.graph.pop(vertex)
            for u in self.graph:
                if vertex in self.graph[u]:
                    self.graph[u].remove(vertex)

    def add_arc(self, arc):
        """
        Given pair of vertices (arc variable) add an arc to the graph
        We consider simple, directed graphs.
        """
        u, v = arc
        self.add_vertex(u)
        self.add_vertex(v)
        if v not in self.graph[u]:
            self.graph[u].append(v)

    def add_edge(self, edge: list):
        """
        Given pair of vertices (edge variable) add an edge to existing graph.
        We consider simple, undirected graphs, as symmetric digraphs without loops.
        """
        u, v = edge
        if u == v:
            raise ValueError("Loops are not allowed!")
        self.add_vertex(u)
        self.add_vertex(v)
        if v not in self.graph[u]:
            self.graph[u].append(v)
        if u not in self.graph[v]:
            self.graph[v].append(u)

    # reading from a file
    @staticmethod
    def from_edges(filename: str, directed = 0):
        """
        Read the graph from file, that in each line contains either
        the description of a vertex (one word) or
        the description of an edge/arc (at least 2 words).
        The resulting graph is returned as a neighbourhood list.
        Variable "filename" contains the whole path to the file.
        """
        graph = Graph()
        file = open(filename, "r")          # open the file to read
        for line in file:                   # for each line of the file
          words = line.strip().split()      # splits the line into words
          if len(words) == 1:               # one word - vertex description
            graph.add_vertex(words[0])
          elif len(words) >= 2:             # at least two words, first two are the edge description
            if directed:
              graph.add_arc([words[0], words[1]])
            else:
              graph.add_edge([words[0], words[1]])
        file.close()
        return graph

    @staticmethod
    def random_graph(n: int, p: float):
        """
        Creates a random graph in G(n, p) model.
        """
        rand_graph = Graph()
        for i in range(1, n + 1):
            rand_graph.add_vertex(i)
            for j in range(1, i):
                if random() < p:
                    rand_graph.add_edge([i, j])
        return rand_graph

    @staticmethod
    def cycle(n: int):
        """
        Creates a cycle C_n on n vertices
        """
        cycle = Graph()
        for i in range(n-1):
          cycle.add_edge([i+1, i+2])
        cycle.add_edge([1, n])
        return cycle


    def Prufer(self):
      """
      Returns the Prufer code of a tree.
      It is necessary that the graph is a tree (it is not checked).
      Result is given as a string (empty for trees on 1 or 2 vertices).
      """
      tr = deepcopy(self.graph)   # copy of a tree, as we destroy it
      code = ""
      for i in range(len(self.graph) - 2):
        for x in sorted(tr):
          if len(tr[x]) == 1:   # least leaf
            break
        v = tr[x][0]            # the unique neighbour of x
        code = code + f"{v} "
        tr[v].remove(x)         # remove x from neighbours of v
        tr.pop(x)               # remove x from the tree
      return code.strip()

    @staticmethod
    def tree_from_Prufer(code: str):
        """
        Creating a tree from a Prufer code.
        """
        tree = Graph()
        clist = [int(x) for x in code.strip().split()]   # code as a list of numbers
        n = len(clist) + 2                  # number of vertices
        vert = [*range(1, n+1)]             # list of numbers 1..n
        for v in vert:
          tree.add_vertex(v)
        for i in range(n-2):
          for x in vert:
            if not x in clist:    # x - least leaf
              break
          v = clist.pop(0)    # remove the first element from the code - the neighbour of x
          tree.add_edge((x, v))
          vert.remove(x)
        tree.add_edge(vert)
        return tree


    def connected_components(self):
      """
      Looks for connected components of undirected graph.
      Returns a list of its vertex-sets.
      Remark: the first element contains the set of all graph vertices
      """
      def DFS(u):
        """
        Deep first search (as internal method).
        """
        for w in self.graph[u]:
          if w not in VT[0]:      # w - not visited yet
            VT[0].add(w)          # already visited
            VT[-1].add(w)         # w - in the last connected component
            DFS(w)

      """
      VT - list of vertex sets VT[i] for i > 0 - is a vertex set of i-th connected component
      VT[0] - is a vertex set of the spanning forest (or during the algorithm list of visited vertices).
      """
      VT = [set([])]
      for v in self.graph:
        if v not in VT[0]:      # v is not visited
          VT[0].add(v)
          VT.append(set([v]))   # statring point of new conected component
          DFS(v)
      return VT

    def preorder(self, v):
        """
        Print vertices of a tree in preorder, starting from vertex v.
        """
        order = []

        def DFS(u, parent=None):
            order.append(u)
            for w in sorted(self.graph.get(u, [])):
                if w != parent:
                    DFS(w, u)

        DFS(v)
        return " ".join(map(str, order))


    def postorder(self, v):
        """
        Print vertices of a tree in postorder, starting from vertex v.
        """
        visited = set()
        order = []

        def DFS(u):
            visited.add(u)
            for w in sorted(self.graph.get(u, [])):
                if w not in visited:
                    DFS(w)
            order.append(u)

        DFS(v)
        return " ".join(map(str, order))


    def InducedSubgraph(self, V):
        """
        Returns a subgraph induced by a set of vertices V.
        """
        sub_dict = {v: [w for w in self.graph[v] if w in V] for v in V}
        subgraph = Graph.from_dict(sub_dict)
        return subgraph

    def ConnectedComponentsGraphs(self):
        """
        Returns a list of Graph objects, each corresponding to one connected component.
        """
        VT = self.connected_components()
        components = []

        for i, vertex_set in enumerate(VT[1:], start=1):
            sub_dict = {
                v: self.graph[v]
                for v in vertex_set
            }
            subgraph = Graph.from_dict(sub_dict)
            components.append(subgraph)

            #print(f"Connected Component {i}:")
            #print(subgraph)

        return components

    def distance(self, v):
      """
      Computes the distance from vertex v to each vertex reachable from it.
      It uses a BFS approach.
      Result is given as a dictionary of distances.
      """
      dist = {v: 0}   # starting point of a dictionary
      queue = [v]
      while len(queue) > 0:
        u = queue.pop(0)
        for w in self.graph[u]:
          if w not in dist:
            dist[w] = dist[u] + 1
            queue.append(w)
      return dist


#new functions------------------------------------------------
    def ConnectedComponentsBFS(self):
        """
        Finds connected components using BFS (non-recursive).
        Returns a list of Graph objects, each corresponding to one connected component.

        self.graph si a dictionary representing a graph
        goal is to split the graph into connected components
        """
        visited = set()
        components = []  #here we store the connected components

        for vertex in self.graph:
            if vertex not in visited: #if a vertex is not visited is because it belongs to a NEW COMPONENT
                component_vertices = set()
                queue = deque([vertex]) #we use queue (FIFO) to explore nodes ---> THIS ENSURES BFS
                visited.add(vertex)
                component_vertices.add(vertex)
                #we create a queue of all the vertex

                #for each vertex we see if the neighbours are visited, we add them to the current component and to the queue for its future neighbours
                while queue:
                    current = queue.popleft()
                    for neighbor in self.graph[current]:
                        if neighbor not in visited:
                            visited.add(neighbor)
                            component_vertices.add(neighbor)
                            queue.append(neighbor)
                      #we ensure that bfs explores every vertex reachable from the initial v

                # after we have visited everything (queue empty) we build subgraph for this component
                sub_dict = {v: [w for w in self.graph[v] if w in component_vertices]
                          for v in component_vertices}
                components.append(Graph.from_dict(sub_dict)) #we add ecah component to the list of components
        return components

    def ConnectedComponentsDFS_without_recursive(self):
        """
        Finds connected components using DFS (non-recursive).
        Returns a list of Graph objects, each corresponding to one connected component.
        """
        visited = set()
        components = []  #here we store the connected components

        for vertex in self.graph:
            if vertex not in visited: #if a vertex is not visited is because it belongs to a NEW COMPONENT
                component_vertices = set()
                stack = [vertex] #instead of queue we use stack (LIFO) to explore nodes ---> THIS ENSURES DFS

                #for each vertex we see if the neighbours are visited, we add them to the current component and to the stack for its future neighbours
                while stack:
                    current = stack.pop()  # DFS uses LIFO
                    if current not in visited:
                        visited.add(current)
                        component_vertices.add(current)
                        for neighbor in self.graph[current]:
                            if neighbor not in visited:
                                stack.append(neighbor)

                # Create subgraph for this component
                sub_dict = {v: [w for w in self.graph[v] if w in component_vertices]
                            for v in component_vertices}
                components.append(Graph.from_dict(sub_dict))

        return components


    def FloydWarshall(self):
        """
        Computes shortest paths between all pairs of vertices using Floyd-Warshall algorithm.
        Returns a distance matrix and a dictionary mapping vertices to indices.
        """
        vertices = list(self.graph.keys())
        n = len(vertices)

        vertex_to_index = {v: i for i, v in enumerate(vertices)} # Create vertex to index mapping

        dist = np.full((n, n), np.inf) # Initialize distance matrix with infinity
        for i in range(n): # Distance from vertex to itself is 0
            dist[i][i] = 0
        # now, for existing edges:
        for u, neighbors in self.graph.items():
            i = vertex_to_index[u]
            for v in neighbors:
                j = vertex_to_index[v]
                dist[i][j] = 1  # Unweighted graph, we say distance is always 1 for each edge

        # Floyd-Warshall algorithm: update shortest paths using intermediate vertices
        # after the loop, it contains the shortest distance between every pair of v
        # because if the current distance is longer than the new calculated, it changes
        for k in range(n):
            for i in range(n):
                for j in range(n):
                    if dist[i][j] > dist[i][k] + dist[k][j]:
                        dist[i][j] = dist[i][k] + dist[k][j] #new distance becomes the current calculated

        return dist, vertex_to_index

    def TopologicalSort(self):
        """
        Returns a list of vertices in topological order:
        it orders vertices so that every edge goes from a vertex earlier in the order to a vertex later in the order.
        """
        in_degree = {v: 0 for v in self.graph} #computes numer of edges coming into a vertex
        for u in self.graph:
            for v in self.graph[u]:
                in_degree[v] += 1

        # starts a queue with in-degree 0 vertices -- they are the first in the topological order
        queue = deque([v for v in self.graph if in_degree[v] == 0])
        topo_order = []

        while queue: #process v in order
            u = queue.popleft()
            topo_order.append(u)

            for v in self.graph[u]:  #for every v that points u we decrease indegree of v and when its 0 we add it to the queue (that stores the topo_order)
                in_degree[v] -= 1
                if in_degree[v] == 0:
                    queue.append(v)

        if len(topo_order) != len(self.graph):  #in case v remain unprocessed is beacuse graph has a cycle --- not possibnle
            raise ValueError("Graph contains cycles, topological sort not possible")

        return topo_order





## Use of code

#LAB 4:


In [30]:

if __name__ == "__main__":
    graph_dict = {
        1: [2, 3],
        2: [1, 3],
        3: [1, 2],
        4: [5],
        5: [4],
        6: [],
        7: [8, 9],
        8: [7],
        9: [7]
    }

    g = Graph(graph_dict)
    print("Graph:")
    print(g)

    # BFS Connected Components
    start_bfs = time.time()
    bfs_components = g.ConnectedComponentsBFS()
    end_bfs = time.time()
    print("BFS Connected Components:")
    for i, comp in enumerate(bfs_components, 1):
        print(f" Component {i}: {list(comp.graph.keys())}")
    print(f"Time taken: {end_bfs - start_bfs:.6f} seconds\n")

    # DFS Connected Components
    start_dfs = time.time()
    dfs_components = g.ConnectedComponentsDFS_without_recursive()
    end_dfs = time.time()
    print("DFS Connected Components:")
    for i, comp in enumerate(dfs_components, 1):
        print(f" Component {i}: {list(comp.graph.keys())}")
    print(f"Time taken: {end_dfs - start_dfs:.6f} seconds\n")

    # Floyd-Warshall
    start_fw = time.time()
    dist_matrix, vertex_to_index = g.FloydWarshall()
    end_fw = time.time()
    print("Floyd-Warshall Distance Matrix:")
    vertices = list(vertex_to_index.keys())
    print("    ", vertices)
    for i, row in enumerate(dist_matrix):
        print(vertices[i], row)
    print(f"Time taken: {end_fw - start_fw:.6f} seconds\n")

    # Create a DAG for topological sort
    dag_dict = {
        1: [2, 3],
        2: [4, 5],
        3: [6],
        4: [7],
        5: [7],
        6: [7],
        7: []
    }

    dag = Graph.from_dict(dag_dict)
    print("DAG graph:")
    print(dag)

    try:
        topo_order = dag.TopologicalSort()
        print(f"Topological order: {topo_order}")
    except ValueError as e:
        print(f"Error: {e}")

    # Test with cyclic graph
    cyclic_dict = {
        1: [2],
        2: [3],
        3: [1]
    }

    cyclic_graph = Graph.from_dict(cyclic_dict)
    print("\nCyclic graph:")
    print(cyclic_graph)

    try:
        topo_order_cyclic = cyclic_graph.TopologicalSort()
        print(f"Topological order: {topo_order_cyclic}")
    except ValueError as e:
        print(f"Expected error for cyclic graph: {e}")

Graph:
1: 2 3
2: 1 3
3: 1 2
4: 5
5: 4
6:
7: 8 9
8: 7
9: 7

BFS Connected Components:
 Component 1: [1, 2, 3]
 Component 2: [4, 5]
 Component 3: [6]
 Component 4: [8, 9, 7]
Time taken: 0.000037 seconds

DFS Connected Components:
 Component 1: [1, 2, 3]
 Component 2: [4, 5]
 Component 3: [6]
 Component 4: [8, 9, 7]
Time taken: 0.000018 seconds

Floyd-Warshall Distance Matrix:
     [1, 2, 3, 4, 5, 6, 7, 8, 9]
1 [ 0.  1.  1. inf inf inf inf inf inf]
2 [ 1.  0.  1. inf inf inf inf inf inf]
3 [ 1.  1.  0. inf inf inf inf inf inf]
4 [inf inf inf  0.  1. inf inf inf inf]
5 [inf inf inf  1.  0. inf inf inf inf]
6 [inf inf inf inf inf  0. inf inf inf]
7 [inf inf inf inf inf inf  0.  1.  1.]
8 [inf inf inf inf inf inf  1.  0.  2.]
9 [inf inf inf inf inf inf  1.  2.  0.]
Time taken: 0.000723 seconds

DAG graph:
1: 2 3
2: 4 5
3: 6
4: 7
5: 7
6: 7
7:

Topological order: [1, 2, 3, 4, 5, 6, 7]

Cyclic graph:
1: 2
2: 3
3: 1

Expected error for cyclic graph: Graph contains cycles, topological sort not po